# Crypto Portfolio Optimization

Goal: Create a machine learning-driven portfolio optimization system for cryptocurrencies, dynamically adjusting allocations to maximize returns and minimize risks based on predictive analytics.

### Top crypto cryptocurrencies 

In [21]:
import requests
import pandas as pd

##### Coingecko API to get the top 25 cryptos 

In [22]:
#&x_cg_demo_api_key=CG-pFKvtbHvwbPpHAPBrgU7a9yc

In [23]:
def fetch_top_coins_tickers(vs_currency='usd', limit=30):
    url = 'https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&ids&x_cg_demo_api_key=CG-pFKvtbHvwbPpHAPBrgU7a9yc'
    params = {
        'vs_currency': vs_currency,
        'order': 'market_cap_desc',
        'per_page': limit,
        'page': 1,
        'sparkline': False,
        'price_change_percentage': '24h'
    }
    response = requests.get(url, params=params)
    if response.status_code != 200:
        print(f"Error fetching data: Status code {response.status_code}")
        return []

    data = response.json()

##Extract tickers
    tickers = [coin['symbol'].upper() for coin in data]
    return tickers

In [24]:
top_coins = fetch_top_coins_tickers(vs_currency='usd', limit=20)
print(top_coins)

['BTC', 'ETH', 'USDT', 'BNB', 'SOL', 'XRP', 'STETH', 'USDC', 'ADA', 'AVAX', 'TRX', 'DOGE', 'LINK', 'DOT', 'MATIC', 'UNI', 'WBTC', 'TON', 'ICP', 'SHIB']


##### yfinance (historical data)

In [25]:
import yfinance as yf


# Specify the symbols for the cryptocurrencies you are interested in
symbols = [f"{coin}-USD" for coin in top_coins]  


df = pd.DataFrame()

# Specify the date range
start_date = '2020-12-31'
end_date = '2024-02-21'

# Fetch the historical data for each symbol
for symbol in symbols:
    print(f"Fetching data for {symbol}...")
    data = yf.download(symbol, start=start_date, end=end_date)
    if not data.empty:
        data['Symbol'] = symbol.replace('-USD', '')  
        df = pd.concat([df, data], axis=0)


df.reset_index(inplace=True)




[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Fetching data for BTC-USD...
Fetching data for ETH-USD...
Fetching data for USDT-USD...
Fetching data for BNB-USD...
Fetching data for SOL-USD...
Fetching data for XRP-USD...
Fetching data for STETH-USD...
Fetching data for USDC-USD...
Fetching data for ADA-USD...
Fetching data for AVAX-USD...



[*********************100%%**********************]  1 of 1 completed

Fetching data for TRX-USD...



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Fetching data for DOGE-USD...
Fetching data for LINK-USD...
Fetching data for DOT-USD...
Fetching data for MATIC-USD...
Fetching data for UNI-USD...
Fetching data for WBTC-USD...
Fetching data for TON-USD...
Fetching data for ICP-USD...
Fetching data for SHIB-USD...


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22799 entries, 0 to 22798
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       22799 non-null  datetime64[ns]
 1   Open       22799 non-null  float64       
 2   High       22799 non-null  float64       
 3   Low        22799 non-null  float64       
 4   Close      22799 non-null  float64       
 5   Adj Close  22799 non-null  float64       
 6   Volume     22799 non-null  int64         
 7   Symbol     22799 non-null  object        
dtypes: datetime64[ns](1), float64(5), int64(1), object(1)
memory usage: 1.4+ MB


### Crypto History Data - data cleansing 

In [27]:
df["Date"] = pd.to_datetime(df["Date"])
df = df.rename(columns={'Volume': 'Volume($)'})

In [28]:
#df['date'] = df['Date'].combine_first(df['index']).combine_first(df['Date'])
#df.drop(['index','Date'], axis=1, inplace=True)
#df=df[['date', 'Symbol', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Market Cap']]        

In [29]:
df.isnull().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume($)    0
Symbol       0
dtype: int64

In [32]:
# Convert the 'Date' column to datetime format for proper sorting
df['Date'] = pd.to_datetime(df['Date'])

# Calculate daily returns for each cryptocurrency
df['Daily Return'] = df.groupby('Symbol')['Close'].pct_change()

df = df[df['Date'] != '2020-12-31']

In [41]:
df.isnull().sum()

Date            0
Open            0
High            0
Low             0
Close           0
Adj Close       0
Volume($)       0
Symbol          0
Daily Return    0
dtype: int64

In [38]:
# Assuming 'df' is the DataFrame containing the data
# First, filter the DataFrame for the date Jan 1, 2024
df_filtered_date = df[df['Date'] == '2024-01-01']

# Now sort this filtered DataFrame based on the 'Close' column to rank the cryptocurrencies
df_sorted_by_close = df_filtered_date.sort_values(by='Close', ascending=False)

# The ranking is implicit in the sorted DataFrame - the first row has the highest closing price and so on
df_sorted_by_close[['Symbol', 'Close']]  # Displaying only the relevant columns for clarity


,Symbol,Close
1096,BTC,44167.332031
19448,WBTC,44080.980469
2243,ETH,2352.327881
7978,STETH,2340.830322
4537,BNB,314.408295
5684,SOL,109.508682
11419,AVAX,41.864464
14860,LINK,15.535809
21612,ICP,12.962581
16007,DOT,8.598925


In [37]:
rows_with_null = df[df.isnull().any(axis=1)]

rows_with_null

,Date,Open,High,Low,Close,Adj Close,Volume($),Symbol,Daily Return
20646,2021-05-10,0.0,750.730469,0.0,428.362305,428.362305,290834080,ICP,NaN
21663,2021-01-02,0.0,0.000000,0.0,0.000000,0.000000,78,SHIB,NaN
21664,2021-01-03,0.0,0.000000,0.0,0.000000,0.000000,2,SHIB,NaN
21665,2021-01-05,0.0,0.000000,0.0,0.000000,0.000000,18,SHIB,NaN
21666,2021-01-09,0.0,0.000000,0.0,0.000000,0.000000,201,SHIB,NaN
...,...,...,...,...,...,...,...,...,...
21753,2021-04-11,0.0,0.000000,0.0,0.000000,0.000000,6995583,SHIB,NaN
21754,2021-04-12,0.0,0.000000,0.0,0.000000,0.000000,17561875,SHIB,NaN
21755,2021-04-13,0.0,0.000000,0.0,0.000000,0.000000,13314013,SHIB,NaN
21756,2021-04-14,0.0,0.000000,0.0,0.000000,0.000000,6349137,SHIB,NaN


In [45]:
df['Symbol'].unique()

array(['BTC', 'ETH', 'USDT', 'BNB', 'SOL', 'XRP', 'STETH', 'USDC', 'ADA',
       'AVAX', 'TRX', 'DOGE', 'LINK', 'DOT', 'MATIC', 'UNI', 'WBTC',
       'TON'], dtype=object)

In [46]:
# removed wrapped coins and the stable coin 'usdc' as they won't help in diversify  my portfolio 
# Define the symbols to exclude
exclude_symbols = ['SHIB', 'ICP','STETH','WBTC','USDC','UNI']

# Filter the DataFrame to exclude the defined symbols
df = df[~df['Symbol'].isin(exclude_symbols)]


In [47]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume($),Symbol,Daily Return
1,2021-01-01,28994.009766,29600.626953,28803.585938,29374.152344,29374.152344,40730301359,BTC,0.012842
2,2021-01-02,29376.455078,33155.117188,29091.181641,32127.267578,32127.267578,67865420765,BTC,0.093726
3,2021-01-03,32129.408203,34608.558594,32052.316406,32782.023438,32782.023438,78665235202,BTC,0.020380
4,2021-01-04,32810.949219,33440.218750,28722.755859,31971.914062,31971.914062,81163475344,BTC,-0.024712
5,2021-01-05,31977.041016,34437.589844,30221.187500,33992.429688,33992.429688,67547324782,BTC,0.063197


In [49]:
df.to_csv('data/top_crypto.csv', index=False)